# Genie Spaces Observability Dashboard

Monitor AI/BI Genie space usage, performance, and costs across your Databricks workspace.

## Data Sources
- **system.access.audit** - Genie events and user actions (365 days retention)
- **system.query.history** - Query execution metrics (365 days retention)
- **system.billing.usage** - Cost tracking (365 days retention)
- **system.billing.list_prices** - Pricing data
- **system.compute.warehouses** - Warehouse metadata

## Query Parameters
Variables declared below control the scope of analysis:
- **date_range_days**: Number of days to analyze (default: 7)
- **workspace_id_filter**: Optional workspace filter (default: '984752964297111')
- **space_id_filter**: Optional Genie space filter (default: '' for all spaces)

### Filter Support Notes:
- Full support: Space lifecycle, user activity, feedback, top users, query performance
- Approximate: Billing (shows warehouse-level costs for warehouses used by space, not exact per-space costs)

In [0]:
%sql
-- Declare SQL variables (Databricks Runtime 14.1+)
DECLARE OR REPLACE VARIABLE date_range_days INT DEFAULT 7;

In [0]:
%sql
DECLARE OR REPLACE VARIABLE workspace_id_filter STRING DEFAULT '984752964297111';


In [0]:
%sql
DECLARE OR REPLACE VARIABLE space_id_filter STRING DEFAULT '';

---
## 1. Space Management

Detailed view of all space lifecycle events (create, clone, trash, delete) with user attribution and space information.

In [0]:
%sql
-- Dataset: space_lifecycle_trends
-- Purpose: Detailed space lifecycle events with user and space information
-- Dashboard Use: Table showing all lifecycle events, can aggregate in dashboard by date/action

SELECT
  event_date,
  event_time,
  action_name,
  CASE
    WHEN action_name = 'createSpace' THEN get_json_object(response.result, '$.space_id')
    ELSE request_params['space_id']
  END as space_id,
  user_identity.email as performed_by,
  workspace_id,
  response.status_code
FROM system.access.audit
WHERE
  service_name = 'aibiGenie'
  AND action_name IN ('createSpace', 'cloneSpace', 'trashSpace', 'deleteSpace')
  AND event_date >= DATEADD(day, -date_range_days, CURRENT_TIMESTAMP())
  AND (workspace_id_filter = '' OR workspace_id = workspace_id_filter)
  AND (
    space_id_filter = '' 
    OR CASE 
        WHEN action_name = 'createSpace' THEN get_json_object(response.result, '$.space_id')
        ELSE request_params['space_id']
      END = space_id_filter
  )
ORDER BY event_time DESC

---
## 2. User Activity & Engagement

Daily engagement metrics tracking conversations, messages, feedback, and interactions

In [0]:
%sql
-- Dataset: user_activity_trends
-- Purpose: Daily activity metrics by action type
-- Dashboard Use: Multi-line time series chart, daily KPI cards

WITH genie_events AS (
  SELECT
    event_date,
    CASE
      WHEN action_name = 'createConversation' THEN 'New Conversation'
      WHEN action_name = 'createConversationMessage' THEN 'New Message'
      WHEN action_name = 'updateConversationMessageFeedback' THEN 'Feedback'
      WHEN action_name = 'createConversationMessageComment' THEN 'Comment'
      WHEN action_name = 'executeMessageQuery' THEN 'Query Re-run'
      ELSE 'Other'
    END as action_category,
    user_identity.email,
    request_params['space_id'] as space_id,
    request_params['conversation_id'] as conversation_id,
    response.status_code
  FROM system.access.audit
  WHERE
    service_name = 'aibiGenie'
    AND action_name IN (
      'createConversation',
      'createConversationMessage',
      'updateConversationMessageFeedback',
      'createConversationMessageComment',
      'executeMessageQuery'
    )
    AND event_date >= DATEADD(day, -date_range_days, CURRENT_TIMESTAMP())
    AND (workspace_id_filter = '' OR workspace_id = workspace_id_filter)
    AND (space_id_filter = '' OR request_params['space_id'] = space_id_filter)
)
SELECT
  event_date,
  action_category,
  COUNT(*) as total_events,
  COUNT(DISTINCT email) as unique_users,
  COUNT(DISTINCT space_id) as unique_spaces,
  COUNT(DISTINCT conversation_id) as unique_conversations,
  COUNT(CASE WHEN status_code != 200 THEN 1 END) as error_count,
  ROUND(COUNT(CASE WHEN status_code != 200 THEN 1 END) * 100.0 / NULLIF(COUNT(*), 0), 2) as error_rate_pct
FROM genie_events
GROUP BY event_date, action_category
ORDER BY event_date, action_category

In [0]:
%sql
-- Dataset: feedback_detail
-- Purpose: Detailed feedback ratings with monitoring URLs
-- Dashboard Use: Table with clickable monitoring URLs

SELECT
  event_time,
  workspace_id,
  user_identity.email as user_email,
  request_params['space_id'] as space_id,
  request_params['conversation_id'] as conversation_id,
  request_params['message_id'] as message_id,
  request_params['feedback_rating'] as feedback_rating,
  CONCAT(
    '/genie/rooms/',
    request_params['space_id'],
    '/monitoring?mc=',
    request_params['conversation_id'],
    '&m=',
    request_params['message_id']
  ) as monitoring_url,
  response.status_code
FROM system.access.audit
WHERE
  service_name = 'aibiGenie'
  AND action_name = 'updateConversationMessageFeedback'
  AND event_date >= DATEADD(day, -date_range_days, CURRENT_TIMESTAMP())
  AND (workspace_id_filter = '' OR workspace_id = workspace_id_filter)
  AND (space_id_filter = '' OR request_params['space_id'] = space_id_filter)
ORDER BY event_time DESC

In [0]:
%sql
-- Dataset: top_users
-- Purpose: Identify power users and adoption leaders
-- Dashboard Use: Ranked table of top users

SELECT
  user_identity.email,
  workspace_id,
  COUNT(*) as total_actions,
  COUNT(DISTINCT request_params['space_id']) as spaces_used,
  COUNT(DISTINCT request_params['conversation_id']) as conversations,
  COUNT(DISTINCT DATE(event_time)) as active_days,
  SUM(CASE WHEN action_name = 'createConversationMessage' THEN 1 ELSE 0 END) as messages_sent,
  SUM(CASE WHEN action_name = 'updateConversationMessageFeedback' THEN 1 ELSE 0 END) as feedback_given
FROM system.access.audit
WHERE
  service_name = 'aibiGenie'
  AND event_date >= DATEADD(day, -date_range_days, CURRENT_TIMESTAMP())
  AND (workspace_id_filter = '' OR workspace_id = workspace_id_filter)
  AND (space_id_filter = '' OR request_params['space_id'] = space_id_filter)
GROUP BY user_identity.email, workspace_id
ORDER BY total_actions DESC
LIMIT 50

In [0]:
%sql
-- Dataset: query_performance_trends
-- Purpose: Detailed query execution records for Genie spaces
-- Dashboard Use: Flexible - can aggregate by day, workspace, space, warehouse, status, etc.

SELECT
    account_id,
    workspace_id,
    statement_id,
    executed_by,
    session_id,
    execution_status,
    compute.warehouse_id AS warehouse_id,
    executed_by_user_id,
    statement_text,
    statement_type,
    error_message,
    client_application,
    client_driver,
    total_duration_ms,
    waiting_for_compute_duration_ms,
    waiting_at_capacity_duration_ms,
    execution_duration_ms,
    compilation_duration_ms,
    total_task_duration_ms,
    result_fetch_duration_ms,
    start_time,
    end_time,
    update_time,
    read_partitions,
    pruned_files,
    read_files,
    read_rows,
    produced_rows,
    read_bytes,
    read_io_cache_percent,
    from_result_cache,
    spilled_local_bytes,
    written_bytes,
    shuffle_read_bytes,
    query_source.genie_space_id AS genie_space_id,
    executed_as_user_id,
    executed_as
FROM system.query.history
WHERE
    client_application = 'Databricks SQL Genie Space'
    AND start_time >= DATEADD(day, -date_range_days, CURRENT_TIMESTAMP())
    AND (workspace_id_filter = '' OR workspace_id = workspace_id_filter)
    AND (space_id_filter = '' OR query_source.genie_space_id = space_id_filter)
ORDER BY start_time DESC

In [0]:
%sql
-- Dataset: warehouse_billing_combined
-- Purpose: Combined billing data (warehouse totals + daily trends)
-- Dashboard Use: Filter by aggregation_level='total' for summary, 'daily' for trends
-- Note: When space_id_filter is used, shows costs for ALL warehouses used by that space (approximate)

WITH genie_warehouses AS (
  SELECT DISTINCT compute.warehouse_id
  FROM system.query.history
  WHERE client_application = 'Databricks SQL Genie Space'
    AND start_time >= DATEADD(day, -date_range_days, CURRENT_TIMESTAMP())
    AND compute.warehouse_id IS NOT NULL
    AND (space_id_filter = '' OR query_source.genie_space_id = space_id_filter)
)
SELECT
  CASE
    WHEN u.usage_date IS NULL THEN 'total'
    ELSE 'daily'
  END as aggregation_level,
  u.usage_date,
  u.usage_metadata.warehouse_id AS warehouse_id,
  COALESCE(w.warehouse_name, u.usage_metadata.warehouse_id) as warehouse_name,
  COUNT(DISTINCT CASE WHEN u.usage_date IS NOT NULL THEN u.usage_date END) as days_active,
  ROUND(SUM(u.usage_quantity), 2) as total_dbu,
  ROUND(SUM(u.usage_quantity * lp.pricing.default), 2) AS cost_usd
FROM system.billing.usage u
JOIN system.billing.list_prices lp
  ON u.sku_name = lp.sku_name
  AND u.cloud = lp.cloud
  AND u.usage_start_time >= lp.price_start_time
  AND (lp.price_end_time IS NULL OR u.usage_start_time < lp.price_end_time)
LEFT JOIN system.compute.warehouses w
  ON u.usage_metadata.warehouse_id = w.warehouse_id
INNER JOIN genie_warehouses gw
  ON u.usage_metadata.warehouse_id = gw.warehouse_id
WHERE u.sku_name LIKE '%SERVERLESS_SQL%'
  AND lp.currency_code = 'USD'
  AND u.usage_date >= DATEADD(day, -date_range_days, CURRENT_TIMESTAMP())
  AND (workspace_id_filter = '' OR u.workspace_id = workspace_id_filter)
GROUP BY GROUPING SETS (
  (u.usage_metadata.warehouse_id, w.warehouse_name),
  (u.usage_date, u.usage_metadata.warehouse_id, w.warehouse_name)
)
ORDER BY aggregation_level, u.usage_date, warehouse_name